In [1]:
import pandas as pd

pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go


from ares_py.class_project import Project
from ares_py.tools.get_ld import get_ld

projects
* name
    * 00_doc
    * 01_2dm
    * 02_crd
    * 03_qcl
    * 04_inv
    * 05_grd
    * 06_mod
    * 99_qgs
        * name_rec.gpkg
            * rec_ert_pt - recorded coordinates in field -ert
            * rec_gen_pt - recorded coordinates - general
        * name_spatial.gpkg
            * crd_plan_ls, crd_plan_pt
            * crd_man_input_pt, crd_man_proc_pt



In [8]:
project_name = "kysuce"

grid_skip = [1, 2, 3, 4, 5, 6]
crs = 8353
res_range = [1, 1000]


atlas_input = dict(
    res_min=[0] * 6,
    res_max=[500] * 6,
    page_heights=[420] * 6,
    map_scales=[500] * 6,
    lvl_frequency=6 * [3],
    z_gap=[20, 15, 15, 25, 25, 25],
)

In [9]:
project_name = "budmerice"

grid_skip = [1, 2, 3, 4, 5, 6]
crs = 8353
res_range = [1, 1000]


atlas_input = dict(
    res_min=[0] * 6,
    res_max=[500] * 6,
    page_heights=[420] * 6,
    map_scales=[500] * 6,
    lvl_frequency=6 * [3],
    z_gap=[20, 15, 15, 25, 25, 25],
)

In [2]:
project_name = "dd_lavka"

grid_skip = [1, 2, 3, 4, 5]
crs = 8353
res_range = [1, 10000]


atlas_input = dict(
    res_min=[100] * 5,
    res_max=[20000] * 5,
    page_heights=[594, 420, 420, 420, 420],
    page_heights_int=[420] + [297] * 4,
    map_scales=[1000] * 5,
    lvl_frequency=5 * [3],
    z_gap=[15, 15, 15, 15, 5],
)

**CREATE PROJECT**

1. Run init project with project name input -> prj class, creates folder structure


In [3]:
prj = Project(project_name, crs=crs, res_range=res_range)

**PROCESS FLAT COORDINATES**

1. Run prj.Process_2dm() - with **flat mode coordinates**


In [ ]:
prj = prj.Process_raw(crd_mode="flat")

projects\dd_lavka\01_input\001.2dm
projects\dd_lavka\01_input\002.2dm
projects\dd_lavka\01_input\003.2dm
projects\dd_lavka\01_input\004.2dm
projects\dd_lavka\01_input\005.2dm


**PROCESS - ert_plan_pt**

1. Draw planed lines - **ert.gpkg - ert_plan_ls**
1. Copy **dtm.tif** into project/qgis folder
1. Run model **ert_ls2pt** in QGIS project and save data manually to **./tmp/tmp_plan.gpkg**
1. Run prj.Process_crd_plan () -> **input/{line}_plan.csv**, **ert.gpkg - ert_plan_pt**
1. Run prj.Process_2dm() -> loads 2dm, adds coordinates from **input/{line}_plan.csv**


In [5]:
prj = prj.Process_crd_plan()
prj = prj.Process_2dm(crd_mode="plan")

projects\dd_lavka\01_input\001.2dm
projects\dd_lavka\01_input\002.2dm
projects\dd_lavka\01_input\003.2dm
projects\dd_lavka\01_input\004.2dm
projects\dd_lavka\01_input\005.2dm


**PROCESS rec coordinates**
1. Run prj.Process_crd_rec() -> **crd_man_input_pt** *(if not exist)*
1. Edit man pt coordinates **ert.gpkg - crd_man_input_pt**
1. Run model rec proc_man  and save to  **tmp/tmp_rec.gpkg**  *(interpolated (1m) coordinates from ert.gpkg - crd_man_input_pt, DTM sampled)*


In [6]:
prj = prj.Process_crd_rec()

Processing rec crd - line 1, 55 entries
Processing rec crd - line 2, 13 entries
Processing rec crd - line 3, 10 entries
Processing rec crd - line 4, 0 entries
Processing rec crd - line 5, 3 entries


c:\Users\adamg\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(


**PROCESS man coordinates**
1. Run prj.Process_crd_man() -> **crd_man_proc_pt** *(coordinates with recalculated ld)*, **input/{line}_man.csv**

In [7]:
prj = prj.Process_crd_man()

c:\Users\adamg\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


**PROCESS 2dm**

1. Set coordinates mode to select coordinates type from csv file:
    + *plan - original planned coordinates*
    + *rec - recorded in field*
    + *man - manual coordinates*
1. Run prj.Process_2dm() -> loads 2dm, adds coordinates from **input/{line}_man.csv**
1. Run prj.Proc_topo_2d() -> exports topo to **ert.gpkg -ert2d_topo_pt, ert2d_topo_ls**


In [8]:
prj = prj.Process_2dm(crd_mode="man")
prj = prj.Proc_topo_2d()

projects\dd_lavka\01_input\001.2dm
projects\dd_lavka\01_input\002.2dm
projects\dd_lavka\01_input\003.2dm
projects\dd_lavka\01_input\004.2dm
projects\dd_lavka\01_input\005.2dm


**Grid data and process contours**
1. Grid grd inputs manually or run prj.Grid_data() to grid and export surfer grids 
    * enter skip list to ignore lines (eg. for long lines)
    * runs gridding on all .csv files in **04_grd/** (except skipped lines)
    * -> **04_grd/{line}_{input_name}.grd

1. Run prj.Export_contours()
    + process contours for all files in **04_grd/** folder
    + -> **ert.gpkg - grd_contours_{type}_ls**


In [11]:
# prj = prj.Grid_data(skip=[grid_skip], cell_size=1)
prj = prj.Export_contours()

Exporting contours - projects\dd_lavka\04_grd\001_r2d.grd
Error exporting contours..
Exporting contours - projects\dd_lavka\04_grd\002_r2d.grd
Error exporting contours..
Exporting contours - projects\dd_lavka\04_grd\003_r2d.grd
Error exporting contours..
Exporting contours - projects\dd_lavka\04_grd\004_r2d.grd
Error exporting contours..
Exporting contours - projects\dd_lavka\04_grd\005_r2d.grd
Error exporting contours..


ValueError: No objects to concatenate

**Process inversion outputs**
1. Do the inversions - export data to **inv/** folder formats:
    + zond - _znd.dat
    + r2d - _topres.dat

1. Run prj.Export_gridding_input() - reads and formats inversion outputs -> **04_grd/{line}.csv**
2. Run prj.Export_gpkg_inv() - exports inversion points to -> **ert.gpkg - inv_{type}_pt**
    + type = r2d / znd
1. Run prj.Export_mask()->  **ert.gpkg - mask_pl**
1. Run 

In [ ]:
prj = prj.Export_gridding_input()

prj = prj.Export_gpkg_inv()
prj = prj.Export_mask()

**Generate atlas_qc:**
1. Enter atlas input: 
    + *res_range_qc - res_min and res_max for qc graphs in Ohmm*
    + *page heights*
    + *map scales*
    + *lvl_frequency = layer frequency for qc graphs 1= all, 2= every 2nd...*
    + *z_gap = offset from top wof atlas extent for grid maps in meters*
1. Run prj.Atlas_qc() -> **ert.gpkg - atlas_qc**
1. Run prj.QC_lines() -> **qgis/{line}.png, qgis/{line}.pgw,** 
1. Run prj.Export_surface_notes()
    + *surface notes polygon on top of atlas window*
    + -> **ert.gpkg - notes_surface_pl** 
    + creates **notes_surface_man_pl* if not exists


In [ ]:
prj = prj.Atlas_qc(atlas_input)
prj = prj.QC_lines()
prj = prj.Export_surface_notes()
pd.DataFrame(prj.atlas_qc)

**EXPORT MODEL TEMPLATE**

1. Run prj.Export_model_template() -> splits mask multipolygon to separate polygons **ert.gpkg - 03_model_pl**

In [ ]:
prj = prj.Export_model_template()

In [ ]:
import geopandas as gpd

data = prj.ert["001"].data

electrodes = pd.DataFrame(data[["c1", "c2", "p1", "p2", "ID_meas"]])
electrodes = electrodes.set_index("ID_meas")

In [ ]:
topo = gpd.read_file(prj.fps["ert"], layer="crd_man_proc_pt")
topo = topo.loc[topo["ID_line"] == 1].set_index("ld")

x = []
y = []
z = []
for col in electrodes.columns:
    x.append(
        pd.merge(electrodes[col], topo["x"], "left", left_on=col, right_index=True)[
            "x"
        ].values
    )
    y.append(
        pd.merge(electrodes[col], topo["y"], "left", left_on=col, right_index=True)[
            "y"
        ].values
    )
    z.append(
        pd.merge(electrodes[col], topo["z0"], "left", left_on=col, right_index=True)[
            "z0"
        ].values
    )

xx = np.column_stack(x)
yy = np.column_stack(y)
zz = np.column_stack(z)

r1 = (0, 2)
r2 = (1, 2)
r3 = (0, 3)
r4 = (1, 3)

dist = []
for c in [r1, r2, r3, r4]:
    dx = (xx[:, c[0]] - xx[:, c[1]]) ** 2
    dy = (yy[:, c[0]] - yy[:, c[1]]) ** 2
    dz = (zz[:, c[0]] - zz[:, c[1]]) ** 2
    dist.append((dx + dy) ** 0.5)
    # dist.append((dx + dy + dz) ** 0.5)

dist = np.column_stack(dist)
dist = 1 / dist
k = 2 * np.pi / (dist[:, 0] - dist[:, 1] - dist[:, 2] + dist[:, 3])

In [ ]:
data["r"] = data["v"] / data["i"]
data["res2"] = data["r"] * data["k2"]
diff = np.clip(data["res"] - data["res2"], -10000, 10000)
data.plot.scatter(data["ld"], diff)

In [ ]:
data["k"] = np.pi * data["n"] * (data["n"] + 1) * data["a"]
data["k"] = data["k"].round()
data.plot.scatter(data["ld"], y=data["k2"] - data["k"])

In [ ]:
from ares_py.qc_lines import fig_qc_lines


def get_levels(df, lvl_frequency):
    df["lab"] = "z= " + np.abs(np.round(df["doi"])).astype(int).astype(str).str.zfill(2)
    cols = ["ld_hor", "res2", "lab"]
    levels = df.groupby("doi")[cols].apply(np.array).to_list()
    levels = levels[::lvl_frequency]
    return levels


lvls = get_levels(data, 3)

ad = prj.atlas_qc.copy().loc[1, :]

fig_qc_lines(prj, lvls, ad, 1)

In [ ]:
import pandas as pd
import geopandas as gpd

df = pd.read_csv(
    "projects/dd_lavka/03_inversion/001_znd_elc.txt", sep="  ", header=None
)
df.columns = ["x", "z0"]
geom = gpd.points_from_xy(df.iloc[:, 0] + 10000, df.iloc[:, 1])
gdf = gpd.GeoDataFrame(df, geometry=geom, crs=8353)
gdf.to_file("tmp/topo_test.gpkg")

In [ ]:
import pandas as pd

df = pd.read_csv("projects/dd_lavka/02_coordinates/001_man.csv")
df = df.loc[df["ld"] % 5 == 0]
df.to_csv("tmp/topo")

In [ ]:
import pandas as pd

df = pd.read_csv(
    r"C:\Users\adamg\OneDrive\01_Processing\ares_py\tmp\dtm_mos.csv", header=None
)
df.columns = ["x", "y", "z"]

df = df.loc[df["z"] != 3.4e38]
df